## Webscraping : site zonebource

In [ ]:
import pandas as pd
import time
#tables = pd.read_html( url, encoding="UTF-8") 

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from  webdriver_manager.chrome import ChromeDriverManager


In [ ]:
def open_browser(url="https://www.zonebourse.com/bourse/actions/_TAap5b/"):
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(url)
    browser.implicitly_wait(5)
    return browser

#####################################################

In [ ]:
df = pd.DataFrame(columns=(["Société", "Cours", "Capitalisation(M$)", "Varia.1janv", "Secteur"]))

In [ ]:
def get_data_set():
    #with pd.ExcelWriter(r"C:\Users\salmen\Desktop\output.xlsx",engine="openpyxl",mode='a') as writer:
    global df
    i = 1
    index = range(50)
    while 1:
        try:
            html=browser.page_source
            src = bytes(html,'utf-8')
            soup = BeautifulSoup(src, 'lxml')
            tables = pd.read_html( str(soup).replace("Ã©","é").replace("Ã","a").replace("¨","è")
                                  .replace("´","u").replace("¯","i"), encoding="utf-8")
            current_df = tables[-1]
            current_df = current_df.drop(0).drop([0,3],axis=1)
            current_df.columns = df.columns
            #current_df.to_excel(writer,sheet_name="Feuil1", startrow=0, startcol=0)
            df = df.append(current_df)
            df.index = range(len(df))
            browser.find_element_by_link_text("Suiv.").click()
            #browser.implicitly_wait(5)
            i+=1
            time.sleep(4)
        except Exception as e:
            print(e)
            print(f'problem in page {i}')
            get_data_set()

In [ ]:
get_data_set()

In [ ]:
df.drop_duplicates(subset ="Société",keep ="first", inplace = True)
df.index = range(len(df))

In [ ]:
df

In [ ]:
with pd.ExcelWriter(r"C:\Users\salmen\Desktop\output.xlsx") as writer:
    df.to_excel(writer, startrow=0, startcol=0)

## Recuperation a partir d'un fichier Excel

In [ ]:
df = pd.read_excel(r"C:\Users\salmen\Desktop\output.xlsx",index_col=0)

In [ ]:
#df['Société'] = df['Société'].str[:-4]
df

## III-Extraction des données financieres pour chaque entreprise
↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓

### INICIALISING DF

In [ ]:
import numpy as np
years = ['2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020','2018', '2019', '2020']
names = ["Chiffre d'affaires","Chiffre d'affaires","Chiffre d'affaires","Résultat d'exploitation (EBIT)","Résultat d'exploitation (EBIT)","Résultat d'exploitation (EBIT)","Résultat net","Résultat net","Résultat net","Dette Nette", "Dette Nette", "Dette Nette","Trésorerie Nette","Trésorerie Nette","Trésorerie Nette","Free Cash Flow","Free Cash Flow","Free Cash Flow","Capitaux Propres","Capitaux Propres","Capitaux Propres","Total Actifs","Total Actifs","Total Actifs","BVPS (Actif net par Action)","BVPS (Actif net par Action)","BVPS (Actif net par Action)"]

In [ ]:
#Initialisation du Data Frame contenant les Etats Financiers
dfa = pd.DataFrame(columns=[names,years])
dfa

Empty DataFrame
Columns: [(Capitalisation, 2018), (Capitalisation, 2019), (Capitalisation, 2020), (Chiffre d'affaires, 2018), (Chiffre d'affaires, 2019), (Chiffre d'affaires, 2020), (Résultat d'exploitation (EBIT), 2018), (Résultat d'exploitation (EBIT), 2019), (Résultat d'exploitation (EBIT), 2020), (Résultat net, 2018), (Résultat net, 2019), (Résultat net, 2020), (Dette Nette, 2018), (Dette Nette, 2019), (Dette Nette, 2020), (Trésorerie Nette, 2018), (Trésorerie Nette, 2019), (Trésorerie Nette, 2020), (Free Cash Flow, 2018), (Free Cash Flow, 2019), (Free Cash Flow, 2020), (Capitaux Propres, 2018), (Capitaux Propres, 2019), (Capitaux Propres, 2020), (Total Actifs, 2018), (Total Actifs, 2019), (Total Actifs, 2020), (BVPS (Actif net par Action), 2018), (BVPS (Actif net par Action), 2019), (BVPS (Actif net par Action), 2020)]
Index: []

[0 rows x 30 columns]

In [ ]:
dfa2 = pd.DataFrame(columns =['name','country'])
dfa2

,name,country


In [ ]:
#entreprises = df['Société'].str[:-4]
#entreprises[2:]

In [ ]:
def get_name_and_country():
    global browser2
    soup = refrech_soup_2()
    t1 = str(soup.find_all('a',itemprop="name")[0])
    t2 = str(soup.find_all('img',style='margin-top:2px;margin-right:5px;',width="40")[0])
    t1 = t1.replace("Ã©","é").replace("Ã","a").replace("¨","è").replace("´","u").replace("¯","i").replace("&amp;","&")
    t2 = t2.replace("Ã©","é").replace("Ã","a").replace("¨","è").replace("´","u").replace("¯","i").replace("&amp;","&")
    a = t1[t1.find('inline;"'):].split("</h1>")[0]
    name = a[a.find('>')+1:]
    b = t2[t2.find('title'):]
    country = b[b.find('"')+1:b.find('w')-2]
    return (name,country)

In [ ]:
def get_current_page(soup):
    soup = refrech_soup()
    num = str(soup.find_all('b')[-1])
    num = num[1:-1]
    num = num[num.find('>')+1:num.find('<')]
    return num

In [ ]:
def get_nums(soup):
    nums = []
    s =list(soup.find_all('a',style='cursor:pointer'))
    for l in s:
        l = str(l)
        b = l[1:-1].find('>')
        e = l[1:-1].find('<')
        nums.append(l[b+2:e+1])
    return nums

In [ ]:
def redirect_to_page(page):
    soup = refrech_soup()
    nums = get_nums(soup)
    current_page = get_current_page(soup)
    if str(page) == current_page:
        return
    elif str(page) in nums:
        browser.find_element_by_link_text(str(page)).click()
        time.sleep(3)
        return
    else:
        while not(str(page) in nums):
            soup = refrech_soup()
            nums = get_nums(soup)
            browser.find_element_by_link_text(nums[-1]).click()
            time.sleep(3)
        browser.find_element_by_link_text(str(page)).click()
        time.sleep(3)
        return

In [ ]:
import re 
def get_or_update_links(soup):
    app_root ='https://www.zonebourse.com'
    reg = re.compile("/cours/action/.{2,50}/")
    len(re.findall(reg,str(soup.find_all('a'))))
    subs = re.findall(reg,str(soup.find_all('a')))
    links = [app_root+sub for sub in subs]
    return links

In [ ]:
def refrech_soup():
    global browser
    html=browser.page_source
    src = bytes(html,'utf-8')
    soup = BeautifulSoup(src, 'lxml')
    return soup

In [ ]:
def refrech_soup_2():
    global browser2
    html=browser2.page_source
    src = bytes(html,'utf-8')
    soup = BeautifulSoup(src, 'lxml')
    return soup

In [ ]:
def get_all_data(i,page):
    global browser
    global browser2
    redirect_to_page(page)
    soup = refrech_soup()
    links = get_or_update_links(soup)
    try:
        j = (i % 50)
        for link in links[j:] :
            dic = {}
            tables = []
            if (link != links[j]):
                browser2.get(link+'/fondamentaux/')
            else:
                browser2 = open_browser(link+'/fondamentaux/')
            (name,country) = get_name_and_country()
            #time.sleep(4)
            #browser.find_element_by_link_text("Plus de Données financières").click()
            time.sleep(2)
            tables = get_single_entreprise_data()
            dic = get_dict_entreprise(tables)
            append_to_df2(name,country,i)
            append_to_df(dic,i)
            #browser.back()
            #time.sleep(2)
            #browser.back()
            #time.sleep(2)
            i+= 1
            if (not(i % 50)):
                #browser.find_element_by_link_text("Suiv.").click()
                page+=1
                time.sleep(2)
                browser2.quit()
                get_all_data(i,page)
    except Exception as e:
        print(e)
        return (i,page,link)

In [ ]:
def get_single_entreprise_data():
    html=browser2.page_source
    src = bytes(html,'utf-8')
    soup = BeautifulSoup(src, 'lxml')
    tables = pd.read_html( str(soup).replace("Ã©","é").replace("Ã","à").replace("¨","è").replace("´","u").replace("¯","i"), encoding="utf-8")
    return tables

In [ ]:
keys = ['Capitalisation',"Chiffre d'affaires","Résultat d'exploitation (EBIT)","Résultat net",
        "Dette Nette","Trésorerie Nette","Free Cash Flow","Capitaux Propres","Total Actifs","BVPS (Actif net par Action)"]
def get_dict_entreprise(tables):
    dic = {}
    for i in range(len(tables)):
        try:
            for key in keys:
                for w in tables[i][0].unique():
                    if isinstance(w,str):
                        if w[:-1] == key or w == key :
                            #print(tables[i].set_index(0).loc[key][:3])
                            x = tables[i].set_index(0).loc[w][:3]
                            if len(x) == 2:
                                x[3] = '-'
                            if len(x) == 1:
                                x[2] ='-'
                                x[3] ='-'
                            dic[key] = x
                            dic[key].index = ['2018','2019','2020']
            if len(dic) != 10:
                for key in keys:
                     if not key in list(dic.keys()):
                            dic[key] = pd.Series(['-','-','-'],index=['2018','2019','2020'])
        except:
            pass
    return dic

In [ ]:
def append_to_df(dic,i):
    global dfa
    data = pd.DataFrame(dic)
    s = data['Capitalisation']
    for column in data.columns[1:]:
        s = s.append(data[column])
    s.index = dfa.columns
    s.name = i
    dfa = dfa.append(s)

In [ ]:
def append_to_df2(name,country,i):
    global dfa2
    s = pd.Series([name,country])
    s.index = dfa2.columns
    s.name = i
    dfa2 = dfa2.append(s)

In [ ]:
#t =get_single_entreprise_data()

In [ ]:
#d = get_dict_entreprise(t)

In [ ]:
#append_to_df(d,152)

In [ ]:
def call(i,page):
    global closed
    global browser
    if closed:
        browser = open_browser()
        #time.sleep(2)
    (x,y,url) = get_all_data(i,page)
    if (x,y) != (i,page):
        browser.quit()
        closed = 1
        time.sleep(2)
        do_get_data(url,x,y)

In [ ]:
def do_get_data(url,i,page):
    global browser
    global closed
    try:
        browser = open_browser(url+'/fondamentaux/')
        dic = {}
        tables = []
        #browser.get(url)
        time.sleep(4)
    except:
        #browser.quit()
        browser.refresh()
        call(i,page)
    time.sleep(2)
    tables = get_single_entreprise_data()
    dic = get_dict_entreprise(tables)
    (name,country) = get_name_and_country()
    append_to_df2(name,country,i)
    append_to_df(dic,i)
    browser.get("https://www.zonebourse.com/bourse/actions/_TAap5b/")
    time.sleep(3)
    i+= 1
    closed =0
    if not(i % 50) :
        page+=1
    call(i,page)

### appel a fonction Mrincipale
↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓

In [ ]:
#Choisir x avant de commancer 
#x = dfa.shape[0]
#x = 1950
x = dfa.tail(1).index[0]+1
p = (x//50)+1
closed = 0
try:
    call(x,p)
except Exception as e :
    print(e)
    #closed = 1
    #browser.quit()
    browser.refresh()
    call(x,p)

<h2 style="color:red">recuperer a partir d un fichier excel</h2>

In [ ]:
#load df from excel file
dfa = pd.read_excel(r"C:\Users\salmen\Desktop\data_final.xlsx",header=[0,1],index_col=0)


In [ ]:
dfa2 = pd.read_excel(r"C:\Users\salmen\Desktop\data2.xlsx",header=0,index_col=0)

### ecrire dans un fichier excel

In [ ]:
#load df in excel file
with pd.ExcelWriter(r"C:\Users\salmen\Desktop\data.xlsx") as writer:
    dfa.to_excel(writer, startrow=0, startcol=0)
#load df in excel file
with pd.ExcelWriter(r"C:\Users\salmen\Desktop\data2.xlsx") as writer:
    dfa2.to_excel(writer, startrow=0, startcol=0)

### voir les données recuperés

In [ ]:
#dfa2['name'] = dfa2['name'].str.replace("&amp;","&")
#dfa2['name']
dfa2.drop_duplicates(keep='first',inplace=True)
#dfa2.index= range(1500,1500+639)

In [ ]:
#last -> 6 -> siemens
#dfa = dfa.loc[dfa2.index.tolist()]
#dfa.index= range(1500,1500+639)
dfa

name-country             Capitalisation             \
                                 0           1           2018       2019   
0                       APPLE INC.  Etats-Unis      1064613.0   993722.0   
1            MICROSOFT CORPORATION  Etats-Unis       757640.0  1026511.0   
2                 AMAZON.COM, INC.  Etats-Unis       734416.0   916154.0   
3                    ALPHABET INC.  Etats-Unis       723465.0   922946.0   
4                     FACEBOOK INC  Etats-Unis       376725.0   585321.0   
...                            ...         ...            ...        ...   
2495  BEIJING TONGRENTANG CO., LTD       Chine        37715.0    38648.0   
2496               KOa HOLDING AS     Turquie        36010.0    51529.0   
2497           HELVETIA HOLDING AG      Suisse         5684.0     6764.0   
2498        CACI INTERNATIONAL INC  Etats-Unis         4163.0     5088.0   
2499              ROYAL VOPAK N.V.    Pays-Bas         5066.0     6168.0   

                Chiffre d'affaires                      \
           2020               2018      2019      2020   
0     1920273.0           265595.0  260174.0  274515.0   
1     1543306.0           110360.0  125843.0  143015.0   
2     1634168.0           232887.0  280522.0  386064.0   
3     1185281.0           136819.0  161857.0  182527.0   
4      778040.0            55838.0   70697.0   85965.0   
...         ...                ...       ...       ...   
2495    32778.0            14209.0   13277.0   12826.0   
2496    53558.0           143248.0  153516.0  183777.0   
2497     4928.0             9073.0    9215.0       NaN   
2498     5441.0             4468.0    4986.0    5720.0   
2499     5349.0             1255.0    1253.0    1190.0   

     Résultat d'exploitation (EBIT)           ... Capitaux Propres  \
                               2018     2019  ...             2020   
0                           70898.0  63930.0  ...          77913.0   
1                           35058.0  42959.0  ...         110317.0   
2                           12421.0  14541.0  ...          77732.0   
3                           26321.0  34231.0  ...         211993.0   
4                           24913.0  28986.0  ...         114672.0   
...                             ...      ...  ...              ...   
2495                         2277.0   1968.0  ...           9533.0   
2496                        11426.0   7931.0  ...          41312.0   
2497                          558.0    671.0  ...           4757.0   
2498                          341.0    378.0  ...           2516.0   
2499                          463.0    539.0  ...              NaN   

     Total Actifs                     Total Passif                      \
             2018      2019      2020         2018      2019      2020   
0        370522.0  352119.0  331203.0          NaN       NaN       NaN   
1        254580.0  272703.0  293934.0          NaN       NaN       NaN   
2        147051.0  193948.0  273222.0          NaN       NaN       NaN   
3        215044.0  238709.0  297762.0          NaN       NaN       NaN   
4         90933.0  115355.0  146346.0          NaN  301962.0       NaN   
...           ...       ...       ...          ...       ...       ...   
2495      19594.0   20698.0   21381.0      56864.0   57263.0  162758.0   
2496     114669.0  138313.0  390722.0          NaN    6027.0       NaN   
2497          NaN       NaN       NaN          NaN    3832.0    3813.0   
2498          NaN       NaN    5315.0       4097.0    6338.0       NaN   
2499       4672.0    9486.0       NaN          NaN       NaN       NaN   

     BVPS (Actif net par Action)                
                            2018   2019   2020  
0                          553.0  509.0  368.0  
1                          108.0  134.0  156.0  
2                          887.0  125.0  186.0  
3                          255.0  293.0  330.0  
4                          292.0  351.0  444.0  
...                          ...    ...    ...  
2495        

In [ ]:
#dfa2[dfa2['name'].duplicated()]
dfa2 

,name,country
5000,ZHOU HEI YA INTERNATIONAL HOLDINGS COMPANY LIM...,Hong Kong
5001,"XENCOR, INC.",Etats-Unis
5002,SHIKUN & BINUI LTD.,Israa«l
5003,"MONALISA GROUP CO.,LTD",Chine
5004,CAP S.A.,Chili


In [ ]:
#merge dfa and dfa2
#dfa2.join(dfa)


In [ ]:
browser = open_browser()

## REGROUPEMENT 

In [ ]:
data={}
data2={}
keys = ['0-499','500-999','1000-1499','1500-2499']

In [ ]:
for key in keys:
    data[key] = pd.read_excel(f'C:/Users/salmen/Desktop/Projet Finance/data zone bource/{key}/data.xlsx',header=[0,1],index_col=0)

In [ ]:
for key in keys:
    data2[key] = pd.read_excel(f'C:/Users/salmen/Desktop/Projet Finance/data zone bource/{key}/data2.xlsx',header=0,index_col=0)

In [ ]:
for key in keys:
    dfa = dfa.append(data[key])

In [ ]:
for key in keys:
    dfa2 = dfa2.append(data2[key])

In [ ]:
import numpy as np

In [ ]:
dfa = dfa.replace('-',np.nan)

In [ ]:
col.append((dfa['Capitaux Propres'] + dfa['Dette Nette']).loc[0])

In [ ]:
col.append(dfa['Capitaux Propres'] + dfa['Dette Nette'])

In [ ]:
dfe = dfa['Capitaux Propres'] + dfa['Dette Nette']

In [ ]:
n = ['Total Passif','Total Passif','Total Passif']